## Import Data

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

######################## PARAMTERS STARTING HERE ################################
# Read the Excel file from the 'Demand' sheet
file_path = "OR113-2_midtermProject_data.xlsx"
df_demand = pd.read_excel(file_path, sheet_name="Demand")
N = df_demand.shape[0] - 1   # -1 because of the first row, +1 for indices' consistency
T = df_demand.shape[1] - 2  # -2 because of the first two columns, +1 for indices' consistency  
print("N:", N, "T:", T)

# Display the dataframe to verify the data
I = np.zeros([N, T])
D = np.zeros([N, T])
I_0 = np.zeros([N])

for i in range(N):
    I_0[i] = df_demand.iloc[i+1, 1]
    for t in range(T):
        D[i, t] = df_demand.iloc[i+1, t+2]

print("I_0:", I_0)
print("D:", D)

# Read the Excel file from the 'In-transit' sheet
df_in_transit = pd.read_excel(file_path, sheet_name="In-transit")
for i in range(N):
    for t in range(df_in_transit.shape[1] - 1):
        I[i, t] = df_in_transit.iloc[i+1, t+1]
print("I:", I)

# Read the Excel file from the 'Shipping cost' sheet
df_shipping_cost = pd.read_excel(file_path, sheet_name="Shipping cost")
J = df_shipping_cost.shape[1] - 1 # -1 because of the first column
df_inventory_cost = pd.read_excel(file_path, sheet_name="Inventory cost")


C = {
    "H": np.zeros([N]),
    "P": np.zeros([N]),
    "V": np.zeros([N, J]),
    "F": np.array([100, 80, 50]),
    "C": 2750,
}
V = np.zeros([N])
V_C = 30
P = np.zeros([N])
BP_percentage = np.zeros([N])
for i in range(N):
    P[i] = df_inventory_cost.iloc[i, 1]
    BP_percentage[i] = df_inventory_cost.iloc[i, 5]
    C["H"][i] = df_inventory_cost.iloc[i, 3]
    C["P"][i] = df_inventory_cost.iloc[i, 2]
    V[i] = df_shipping_cost.iloc[i, 3]
    for j in range(J):
        if j == J - 1:
            C["V"][i, j] = 0
        else:
            C["V"][i, j] = df_shipping_cost.iloc[i, j+1]

print("P:", P)
print("BP_percentage:", BP_percentage)
print("C:", C)
print("V:", V)
T_lead = np.array([1, 2, 3]) # T_j

N: 10 T: 6
I_0: [800. 600. 425. 350. 400. 524. 453. 218. 673. 200.]
D: [[138.  55. 172. 194.  94. 185.]
 [190. 101.  68. 185.  13. 136.]
 [ 79. 179.  21.  49. 199. 200.]
 [142. 103.  78. 131. 146. 155.]
 [ 35.  62.  83.  90. 197.  49.]
 [ 91.  95. 107. 127. 116. 183.]
 [105. 164.  19. 116. 119. 175.]
 [ 37. 155.  10.  77. 168.  32.]
 [108. 185. 188. 176.  81. 172.]
 [ 46. 178. 162. 200. 154. 199.]]
I: [[  0.   0.   0.   0.   0.   0.]
 [ 48.   0.   0.   0.   0.   0.]
 [  0.  20.   0.   0.   0.   0.]
 [153.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [ 18.  23.   0.   0.   0.   0.]
 [ 28.  45.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [109.  34.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]]
P: [10700.  5000. 26900. 19600.  6900. 31500. 14100. 16400. 33000. 18600.]
BP_percentage: [0.  0.7 0.1 1.  1.  0.3 0.6 0.2 0.1 0.5]
C: {'H': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'P': array([5000., 2000., 9000., 9000., 2000., 9000., 7000., 500

## Invoke Solver

In [2]:
import gurobipy as gp
from gurobipy import GRB

# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

# Create the Gurobi model
model = gp.Model("InventoryManagement")

# Set error parameter
model.setParam('MIPGap', 0.0)

# Define sets
S_I = range(N)  # Products i in {0,  ..., N-1}
S_T = range(T)  # Time periods t in {0, ..., T-1}
S_J = range(J)  # Shipping methods j in {0, ..., J-1}

# Variables
x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
y = model.addVars(S_J, S_T, vtype=GRB.BINARY, name="y")  # Binary for shipping method y_jt
z = model.addVars(S_T, vtype=GRB.INTEGER, name="z")  # Number of containers z_t
w = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="w")
s = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="g")


### Objective function ###
# Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
purchasing_and_shipping_cost = gp.quicksum(
    (C["P"][i] + C["V"][i, j]) * x[i, j, t]
    for i in S_I for j in S_J for t in S_T
) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)
shortage_cost = gp.quicksum((P[i] - C["P"][i]) * w[i, t] for i in S_I for t in S_T)

model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost + shortage_cost, GRB.MINIMIZE)

#### Constraints ###
# Inventory balance
J_in_inventory = np.array([1, 2, 3, 3, 3, 3])

for i in S_I:
    for t in S_T:
        # Compute the in-transit quantity arriving at time t
        in_inventory = 0
        for j in range(J_in_inventory[t]):
            in_inventory += x[i, j, t - T_lead[j] + 1]
        # Add the constraint for inventory balance
        if t == 0:
            model.addConstr(v[i, t] == I_0[i] - s[i, t] + I[i, t] + in_inventory , name=f"InvBalance_{i}_{t}")
        else:
            model.addConstr(v[i, t] == v[i, t-1] - s[i, t] + I[i, t] + in_inventory, name=f"InvBalance_{i}_{t}")

# Relate order quantity and shipping method
M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
for j in S_J:
    for t in S_T:
        model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

# Container constraint
for t in S_T:
    model.addConstr(
        gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
        name=f"Container_{t}"
    )

# v, w, s constraints
for i in S_I:
    for t in S_T:
        model.addConstr(w[i,t] >= D[i,t] - s[i,t], name=f"w_{i}_{t}")
        model.addConstr(w[i, t] >= 0, name=f"w_{i}_{t}")
        if t == 0:
            model.addConstr(s[i, t] <= I_0[i], name=f"s_{i}_{t}")
        else:
            model.addConstr(s[i, t] <= v[i, t-1], name=f"s_{i}_{t}")


# Non-negativity and binary constraints
for i in S_I:
    for j in S_J:
        for t in S_T:
            model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
for j in S_J:
    for t in S_T:
        model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
for t in S_T:
    model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

# Optimize the model
model.optimize()


df = pd.DataFrame()


### visualize results ###
if model.status == GRB.OPTIMAL:
    original_objective_value = float(16890367.02)
    new_objective_value = model.objVal
    optimility_gap = ((original_objective_value - new_objective_value) / original_objective_value) * 100
    print(f"Original objective value: {original_objective_value}")
    print(f"\nOptimal objective value: {new_objective_value}\n")
    print(f"optimility gap: {optimility_gap :.10f}%")

    results = {}
    for i in S_I:
        # collect, for each time t, the x-values for all shipping methods j
        data = {}
        data["Demand"]               = [D[i, t]        for t in S_T]
        data['Sold']               = [s[i, t].x      for t in S_T]
        data["Shortage"]                = [w[i, t].x      for t in S_T]
        for j in S_J:
            if j == 0:
                data[f"Order_qty_Express"] = [x[i, j, t].x for t in S_T]
            elif j == 1:
                data[f"Order_qty_Air"] = [x[i, j, t].x for t in S_T]
            elif j == 2:
                data[f"Order_qty_Shipping"] = [x[i, j, t].x for t in S_T]
            
        data["Inventory_in_Transit"] = [I[i, t]        for t in S_T]
        data["Ending_inventory"]     = [v[i, t].x      for t in S_T]

        # build DataFrame with periods as rows…
        df_i = pd.DataFrame(
            data,
            index=["March", "April", "May", "June", "July", "August"]
        )

        # …then transpose so metrics are rows, periods are columns
        df_i = df_i.T
        # df_i.index.name   = f"Product_{i+1}_Metric"
        df_i.columns.name = "Period"

        results[i+1] = df_i

        width = 10  # Adjust this to fit your desired spacing
        print("===" * 8 + f"Procurement Planning for Product {i+1}" + "===" * 8)
        print(f"{'Initial inventory for product ' + str(i+1):<35}{I_0[i]:>{width}.2f}")
        print("-" * 80)
        print(df_i, "\n")

        df = pd.concat([df, df_i], axis=0)

    # Write the results to an Excel file
    with pd.ExcelWriter("p1_results.xlsx") as writer:
        df.to_excel(writer, sheet_name="p1_results")

    # print("All tables (including Demand & In-Transit) written to inventory_results_full.xlsx")
else:
    print("No optimal solution found.")

Set parameter Username
Set parameter LicenseID to value 2636968
Academic license - for non-commercial use only - expires 2026-03-15
Set parameter MIPGap to value 0


GurobiError: Element 0 of a double array is Nan or Inf.